In [1]:
import bz2
import collections
import os
import tensorflow as tf
import numpy as np
#import lazy_property
from helpers import download
from lxml import etree
from attrdict import AttrDict
from EmbeddingModel import EmbeddingModel
from RawDataParser import Wikipedia
from withableWriter import withableWriter

In [3]:
def skipgrams(pages, max_context):
    """Form training pairs according to the skip-gram model."""
    for words in pages:
        for index, current in enumerate(words):
            context = np.random.randint(1, max_context)
            for target in words[max(0, index - context): index]:
                yield current, target
            for target in words[index + 1: index + context + 1]:
                yield current, target


def batched(iterator, batch_size):
    """Group a numerical stream into batches and yield them as Numpy arrays."""
    while True:
        data = np.zeros(batch_size)
        target = np.zeros(batch_size)
        for index in range(batch_size):
            data[index], target[index] = next(iterator)
        yield data, target
    
params = AttrDict(
    vocabulary_size=1000000,
    max_context=10,
    embedding_size=200,
    contrastive_examples=10,
    learning_rate=0.5,
    momentum=0.5,
    batch_size=1000,
)  

data = tf.placeholder(tf.int32, [None])
target = tf.placeholder(tf.int32, [None])
model = EmbeddingModel(data, target, params)

corpus = Wikipedia(
    'https://dumps.wikimedia.org/enwiki/20171120/enwiki-20171120-pages-meta-current1.xml-p10p30303.bz2',
    'wikipedia',
    params.vocabulary_size)
examples = skipgrams(corpus, params.max_context)
batches = batched(examples, params.batch_size)

cost_summary = tf.summary.scalar(tensor = model.cost, name = 'cost')


Read vocabulary


In [4]:
def similarity(vec1, vec2):
    return np.dot(vec1,vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

def nearestNeighbours(vectors, target, k):
    distances = []
    vectors = np.stack(vectors)
    for i in range(len(vectors)):
        distances.append(similarity(vectors[i], target))
    indices = [i[0] for i in sorted(enumerate(distances), key=lambda x:x[1], reverse=True)]
    distances= sorted(distances)
    return indices[:k], distances[:k]    

vectors = np.load('embeddings_626495_1999999.npy')



In [7]:
vocabulary_size= 999999
j = corpus.encode('city')
indices, _ = nearestNeighbours(vectors, vectors[j], 100)
nn = [corpus.decode(min(i, vocabulary_size)) for i in indices]
print(nn)

print('\n')

j = corpus.encode('york')
indices, _ = nearestNeighbours(vectors, vectors[j], 100)
nn = [corpus.decode(min(i, vocabulary_size)) for i in indices]
print(nn)

['city', 'of', '<', '.', 'that', 'cite', 'web', 'and', 'pages', 'b', 'ref', 'for', '(', 'p', 'the', 'war', 'first', 'www', 'is', 'name', 'publisher', 'from', 'was', 'in', 'which', ',', ':', 'by', 'he', 'nbsp', 'on', 'as', 'http', 'his', 'to', 'had', 'than', ')', 'are', 'an', 'url', 's', '?', 'ndash', 'journal', 'title', 'com', 'at', 'with', 'a', 'd', 'one', 'other', 'https', 'most', 'volume', 'accessdate', 'org', 'last', 'e', 'book', 'their', 'it', 'but', '!', 'year', 'some', 'author', 'were', 'news', 'n', 'can', 'html', 'this', 'have', 'no', 'y', 'date', 'isbn', 'de', 'c', 'world', 'has', 'style', 'american', 'use', 'they', 'when', 'university', 'i', 'its', 'new', 'john', 'or', 'order', 'after', 'states', 'br', 'august', 'not']


['york', 'in', '.', 'd', ',', 'ref', 'name', 'web', 'last', 'url', 'cite', '<', 'to', 'american', 'the', 'with', 'book', 'language', 'by', 'first', ')', 'his', 'of', 'also', '(', 'new', 'that', ':', 'he', 'a', 'at', 'it', 'c', 'not', 'be', 'title', 'pages', '

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
average = collections.deque(maxlen=100)
for index, batch in enumerate(batches):
    feed_dict = {data: batch[0], target: batch[1]}
    cost, _ = sess.run([model.cost, model.optimize], feed_dict)
    average.append(cost)
    print('{}: {:5.1f}'.format(index + 1, sum(average) / len(average)))

embeddings = sess.run(model.embeddings)
np.save('/home/pietro/wikipedia/embeddings.npy', embeddings)

In [15]:
distance(np.stack(vectors)[0], np.stack(vectors)[0])

0.0

In [5]:
ind, val = sorted([1,4,2,6,3])

ValueError: too many values to unpack (expected 2)

In [22]:
indices

[858, 2, 0, 1, 6, 3, 75, 8, 63, 7, 37, 46, 113, 99, 202, 126, 132, 69, 4, 11]